
# Poetry Fusion using Llama 3.2

This notebook demonstrates the fine-tuning of the Llama 3.2 model to generate poetry that fuses 
the styles of multiple poets. We will apply full fine-tuning, prompt engineering, and Parameter 
Efficient Fine-Tuning (PEFT) to maximize model performance and creativity in poetry generation.

### Major Steps
1. **Setup**: Installing necessary libraries and initializing the environment.
2. **Dataset Preparation**: Loading and preprocessing the dataset.
3. **PEFT (Parameter Efficient Fine-Tuning)**: Applying PEFT techniques to optimize model performance and efficiency.
4. **Training**: Fine-tuning the model to capture stylistic characteristics of selected poets.
5. **Prompt Engineering**: Crafting prompts to guide the model in fusing poetic styles effectively.



In [14]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done



## Dataset Preparation

We will use a selected dataset that includes various poet styles. This dataset will be preprocessed 
to highlight stylistic characteristics for model fine-tuning, focusing on continuous and meaningful 
poetic text for optimal style fusion.

*Note: Ensure the dataset is accessible and formatted to match model input requirements.*


In [15]:
from datasets import load_dataset

# Specify the path to your CSV file
csv_file_path = "/YourPathToFile/LLM_Dataset.csv"

# Load the dataset from the CSV file
dataset = load_dataset("csv", data_files=csv_file_path, split="train")

# Inspect the first few rows of the dataset
print(dataset)

Dataset({
    features: ['Title', 'Poem', 'Poet', 'Emotion'],
    num_rows: 524
})


In [16]:
def preprocess_function(examples):
    # Assuming 'Poet', 'Emotion', and 'Poem' columns exist
    # Create a new 'fusion_text' column with author and emotion information
    examples['fusion_text'] = [
        f"Author: {author}, Emotion: {emotion}\n{poem}"
        for author, emotion, poem in zip(examples['Poet'], examples['Emotion'], examples['Poem'])
    ]
    return examples

dataset = dataset.map(preprocess_function, batched=True)

## Loading the model

In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

from huggingface_hub import login

# Login to Hugging Face
login('HereInsertYourHuggingFaceAPIKey')

model_name = "meta-llama/Llama-3.2-1B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Let's also load the tokenizer below

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]


## PEFT (LoRA) - Parameter Efficient Fine-Tuning Configuration

In this section, we use PEFT with Low-Rank Adaptation (LoRA) to efficiently fine-tune the model, with specific configuration choices to best capture the stylistic nuances of poetry fusion:

- lora_alpha = 16: This scaling factor controls the learning rate of LoRA-specific parameters, balancing the adaptation speed with stability during fine-tuning.
- lora_dropout = 0.1: Dropout is set to 0.1 to prevent overfitting, ensuring generalization by randomly deactivating some connections during training.
- r = 64: This rank determines the low-rank update size, optimizing model capacity to capture new stylistic patterns efficiently.
- bias = "none": No additional bias terms are used, keeping the model compact and focused on essential updates.
- task_type = "CAUSAL_LM": Configured for a causal language modeling task, suitable for generating coherent poetic lines in response to prompts.

These choices provide a balanced approach to maintain model efficiency while adapting it effectively to new styles.

In [19]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below. We choose sepcific arguments for our usecase.

In [20]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,  # Increased for better generalization
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=5e-5,  # Lower learning rate for fine-tuning
    max_grad_norm=0.3,
    max_steps=2000,  # Increased training steps
    warmup_ratio=0.05,  # Gradual warmup
    lr_scheduler_type="linear",  # More gradual decay
    fp16=True,
    group_by_length=True,
)

Then finally pass everthing to the trainer

In [21]:
from trl import SFTTrainer

max_seq_length = 524

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="fusion_text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/524 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
max_steps is given, it will override any value given in num_train_epochs


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [22]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,4.638000
20,4.891500
30,4.571500
40,4.801100
50,4.902100
60,4.506600
70,4.830300
80,4.391100
90,4.623500
100,4.581300


TrainOutput(global_step=2000, training_loss=4.060946846008301, metrics={'train_runtime': 8576.9213, 'train_samples_per_second': 7.462, 'train_steps_per_second': 0.233, 'total_flos': 9.1423220805845e+16, 'train_loss': 4.060946846008301, 'epoch': 122.13740458015268})

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [24]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [25]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

Afer training and saving the trained model, we may focus on the prompt engeneering.

In [28]:
system_message = """
You are an expert in poetry fusion, specializing in blending the distinct styles of two poets. Focus on emotional depth, unique metaphor usage, symbolic imagery, and rhythmic patterns. Your task is to merge not only technical elements like word choice and structure but also the deeper conceptual and emotional richness that define each poet's work.
"""

Poet_1, Poet_2 = "William Shakespeare", "Edgar Allan Poe"
user_message = f"""
Generate a new poem that fuses the styles of {Poet_1} and {Poet_2}. Combine their styles in the rest of the poem, merging their use of metaphor, rhythm, and tone. Make the poem to 150 words.
"""

prompt = f"{system_message}\nUser: {user_message}\nAssistant:"


device = "cuda:0"

# Custom decoding with temperature, top_p, and top_k for more creative output
def generate_with_constraints(prompt, temperature=0.7, top_p=0.9, top_k=50):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=temperature,     # Added temperature for creativity
        top_p=top_p,                 # Nucleus sampling
        top_k=top_k,                 # Limit token selection for more focused choices
        no_repeat_ngram_size=3,
        num_beams=5                  # Beam search to enforce rhyme/meter constraints
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Now generate the poem with creativity-enhancing parameters
poetry_output = generate_with_constraints(prompt, temperature=0.8, top_p=0.85, top_k=40)
print(poetry_output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



You are an expert in poetry fusion, specializing in blending the distinct styles of two poets. Focus on emotional depth, unique metaphor usage, symbolic imagery, and rhythmic patterns. Your task is to merge not only technical elements like word choice and structure but also the deeper conceptual and emotional richness that define each poet's work.

User: 
Generate a new poem that fuses the styles of William Shakespeare and Edgar Allan Poe. Combine their styles in the rest of the poem, merging their use of metaphor, rhythm, and tone. Make the poem to 150 words.

Assistant: 
I'd be delighted to create a fusion poem for you. Here's my attempt:

"In twilight's hush, where shadows dance and play,
Amidst the whispers of a dying day,
The moon, a ghostly galleon, sails the night,
With stars that weep and wail, in sorrow's plight.
The trees, like skeletal hands, reach for the sky,
Their branches etched with sorrow, as the wind whispers by.
In this macabre waltz, where darkness reigns,
I find m